In [0]:
%run "./Configuration"

In [0]:
# Define the metadata widget:
dbutils.widgets.text("metaData", "")

# Save the value as a JSON object:
metadata = json.loads(dbutils.widgets.get("metaData"))

In [0]:
if metadata['entityName'] == 'AdventureWorks':
    # Define the url to the CSV file:
    csv_url = metadata['connectionString']
    # Get the CSV file content as a string:
    response = requests.get(csv_url)
    # Define temp path in DBFS:
    temp_path = '/dbfs/tmp/temp_csv.csv'
    # Write the string to a csv file in DBFS:
    with open(temp_path, 'wb') as f:
        f.write(response.content)
    
    # Create a DataFrame from the CSV file:
    df = spark.read.csv('dbfs:/tmp/temp_csv.csv', header=True, inferSchema=True)

In [0]:
folder_path = datetime.datetime.now().strftime("%Y/%m/%d")

In [0]:
# Check data exists:
if df.count() > 0:
    # Set location for parquet:
    location = RAW + f"/{metadata['entityName']}/{folder_path}/"

    # Write parquet to data lake:
    df.write.mode('overwrite').format('parquet').option('header', 'true').save(location)